In [17]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, auc
from sklearn.preprocessing import OrdinalEncoder
from sklearn.neural_network import MLPClassifier

In [18]:
df = pd.read_csv('kddcup99_csv.csv', sep=',')

# KDD Cup 99

Dataset obsahuje zaznamenanou komunikaci v siti ktera mela simulovat realne provoz ve vojenske siti. Kazdy zaznam obsahuje informace rozparsovany tcp packet. Z jednotlivych atributu chceme zjistit, zda se jedna o dobrou, ci spatnou komunikaci (utok). Nasledne se tento utok deli na dalsi kategorie.

* DOS: denial-of-service, e.g. syn flood;
* R2L: unauthorized access from a remote machine, e.g. guessing password;
* U2R: unauthorized access to local superuser (root) privileges, e.g., various ``buffer overflow’’ attacks;
* probing: surveillance and other probing, e.g., port scanning.

Podivame se na par zakladnich atributu.

In [19]:
df.dtypes

duration                         int64
protocol_type                   object
service                         object
flag                            object
src_bytes                        int64
dst_bytes                        int64
land                             int64
wrong_fragment                   int64
urgent                           int64
hot                              int64
num_failed_logins                int64
logged_in                        int64
lnum_compromised                 int64
lroot_shell                      int64
lsu_attempted                    int64
lnum_root                        int64
lnum_file_creations              int64
lnum_shells                      int64
lnum_access_files                int64
lnum_outbound_cmds               int64
is_host_login                    int64
is_guest_login                   int64
count                            int64
srv_count                        int64
serror_rate                    float64
srv_serror_rate          

In [34]:
df.dst_host_rerror_rate.value_counts()

0.00    458791
1.00     26040
0.01      1596
0.02       932
0.04       801
         ...  
0.63        19
0.34        18
0.46        15
0.39        14
0.79        13
Name: dst_host_rerror_rate, Length: 101, dtype: int64

## Nas target atr. neboli vyhodnocene pripojeni
Abychom byli schopni provest klasifikaci, budeme muset objekt filtrovat, "normal" bude good connection a zbytek bad connection.

respektive 0-Good, 1-Bad 

In [20]:
df.label.unique()

array(['normal', 'buffer_overflow', 'loadmodule', 'perl', 'neptune',
       'smurf', 'guess_passwd', 'pod', 'teardrop', 'portsweep', 'ipsweep',
       'land', 'ftp_write', 'back', 'imap', 'satan', 'phf', 'nmap',
       'multihop', 'warezmaster', 'warezclient', 'spy', 'rootkit'],
      dtype=object)

In [29]:
df.label.value_counts()

smurf              280790
neptune            107201
normal              97277
back                 2203
satan                1589
ipsweep              1247
portsweep            1040
warezclient          1020
teardrop              979
pod                   264
nmap                  231
guess_passwd           53
buffer_overflow        30
land                   21
warezmaster            20
imap                   12
rootkit                10
loadmodule              9
ftp_write               8
multihop                7
phf                     4
perl                    3
spy                     2
Name: label, dtype: int64

## Protokoly pripojeni

In [21]:
df.protocol_type.unique()

array(['tcp', 'udp', 'icmp'], dtype=object)

## Servisy

In [22]:
df.service.unique()

array(['http', 'smtp', 'finger', 'domain_u', 'auth', 'telnet', 'ftp',
       'eco_i', 'ntp_u', 'ecr_i', 'other', 'private', 'pop_3', 'ftp_data',
       'rje', 'time', 'mtp', 'link', 'remote_job', 'gopher', 'ssh',
       'name', 'whois', 'domain', 'login', 'imap4', 'daytime', 'ctf',
       'nntp', 'shell', 'IRC', 'nnsp', 'http_443', 'exec', 'printer',
       'efs', 'courier', 'uucp', 'klogin', 'kshell', 'echo', 'discard',
       'systat', 'supdup', 'iso_tsap', 'hostnames', 'csnet_ns', 'pop_2',
       'sunrpc', 'uucp_path', 'netbios_ns', 'netbios_ssn', 'netbios_dgm',
       'sql_net', 'vmnet', 'bgp', 'Z39_50', 'ldap', 'netstat', 'urh_i',
       'X11', 'urp_i', 'pm_dump', 'tftp_u', 'tim_i', 'red_i'],
      dtype=object)

### Uz ted vime, ze budeme muset prevest objekty na enums aby se s nimi nas model vyporadal

In [30]:
df.flag.value_counts()

SF        378439
S0         87007
REJ        26875
RSTR         903
RSTO         579
SH           107
S1            57
S2            24
RSTOS0        11
S3            10
OTH            8
Name: flag, dtype: int64

In [35]:
df.describe(exclude=np.number)

,protocol_type,service,flag,label
count,494020,494020,494020,494020
unique,3,66,11,23
top,icmp,ecr_i,SF,smurf
freq,283602,281400,378439,280790


## Chybejici atributy ?
Zadne nejsou 

In [36]:
df.apply(lambda x: x.isna().sum()).sort_values(ascending=False)

duration                       0
dst_host_count                 0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_host_rate             0
dst_host_srv_count             0
protocol_type                  0
dst_host_same_srv_rate         0
dst_host_diff_srv_rate         0
dst_host_same_src_port_rate    0
dst_host_srv_diff_host_rate    0
dst_host_serror_rate           0
dst_host_srv_serror_rate       0
dst_host_rerror_rate           0
dst_host_srv_rerror_rate       0
count                          0
is_guest_login                 0
is_host_login                  0
lnum_outbound_cmds             0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent    